In [1]:
from pathlib import Path
import time
import zmq
import json

In [2]:
class StoredDict():
    def __init__(self,bucket='default'):
        self.data = {}
        self.file = Path(bucket+'.json')
        if self.file.exists():
            self.data = json.loads(self.file.read_text())
    def update(self, other):
        self.data.update(other)
        self.file.write_text(json.dumps(self.data,indent=4))
    def delete(self,key):
        if key in self.data:
            del self.data[key]
            self.file.write_text(json.dumps(self.data,indent=4))
    def toPacket(self):
        return self.data
    
    

In [ ]:
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5555")
data = {
    'default':StoredDict('default')
}
print('loop running')
while True:
    #  Wait for next request from client
    message = socket.recv()    
    if len(message)>1:
        bucket,message  = message.decode('utf-8').split(':',maxsplit=1)
        if bucket in data:
            if message.startswith('del'):
                data[bucket].delete(message[4:])
            elif len(message)>1:
                message = json.loads(message)
                data[bucket].update(message)
        else:
            data[bucket] = StoredDict(bucket)
    
        socket.send_json(data[bucket].toPacket())
    else:
        socket.send_json({k:data[k].toPacket() for k in data})
    print()


loop running
b'1'

b'1'

b'1'

b'Report:1'
Report
1

b'Report:del lastrun_time'
Report
del lastrun_time

b'Report:{"start_time": 1659989518}'
Report
{"start_time": 1659989518}

b'1'

b'1'

b'Report:{"test": 0}'
Report
{"test": 0}

b'Report:{"completion": 0}'
Report
{"completion": 0}

b'Report:{"completion": 1}'
Report
{"completion": 1}

b'Report:{"completion": 2}'
Report
{"completion": 2}

b'Report:{"completion": 3}'
Report
{"completion": 3}

b'Report:{"completion": 4}'
Report
{"completion": 4}

b'Report:{"completion": 5}'
Report
{"completion": 5}

b'Report:{"completion": 6}'
Report
{"completion": 6}

b'Report:{"completion": 7}'
Report
{"completion": 7}

b'Report:{"completion": 8}'
Report
{"completion": 8}

b'Report:{"completion": 9}'
Report
{"completion": 9}

b'Report:{"completion": 10}'
Report
{"completion": 10}

b'Report:{"completion": 11}'
Report
{"completion": 11}

b'Report:{"completion": 12}'
Report
{"completion": 12}

b'Report:{"completion": 13}'
Report
{"completion": 13}

b'Rep